In [ ]:
import functools
from datetime import datetime

from split_query.expressions import Attribute, Ge, Le, Gt, Lt, And, Or, Eq
from split_query.simplify import simplify_tree
from split_query.truth_table import expand_dnf
from split_query.domain import simplify_domain
from interface import DataSet
from converters import convert_expression, soql, in_to_or, map_to_ids

In [ ]:
dataset = DataSet('Pedestrians', [Attribute(n) for n in ['datetime', 'hourly_count', 'sensor']], None)

In [ ]:
filtered = dataset[
    dataset.datetime.between(datetime(2016, 6, 1), datetime(2017, 7, 1))][
    dataset.sensor.isin(['bourke', 'grattan'])]

In [ ]:
def map_to_year(obj):
    ''' Should generalise map_to_ids to do this: generally take a function or
    map which converts values for a particular attribute. '''
    if isinstance(obj, Ge) and obj.attribute == Attribute('datetime'):
        return Or([
            And([Eq(Attribute('year'), obj.value.year), Ge(Attribute('month'), obj.value.month)]),
            Gt(Attribute('year'), obj.value.year)])
    if isinstance(obj, Le) and obj.attribute == Attribute('datetime'):
        return Or([
            And([Eq(Attribute('year'), obj.value.year), Le(Attribute('month'), obj.value.month)]),
            Lt(Attribute('year'), obj.value.year)])
    return obj

expression = filtered.expr
expression = convert_expression(expression, hook=map_to_year)
attribute_map = {
    Attribute('sensor'): (Attribute('sensor_id'), {
        'bourke': 27, 'grattan': 48})}
expression = convert_expression(expression, hook=functools.partial(
    map_to_ids, attribute_map=attribute_map))
expression = convert_expression(expression, hook=in_to_or)
expression = convert_expression(expression, hook=soql)
expression